In [ ]:
import spacy

In [ ]:
nlp = spacy.load('vfbr_persons_model')

In [ ]:
for x in VfbEntry.objects.all():
    doc = nlp(x.vollregest)
    try:
        if len(doc.ents) > 0:
            if len(doc.ents[0].text) > 5:
                pers, _ = Person.objects.get_or_create(legacy_id=doc.ents[0].text[:299])
            else:
                pers = None
        else:
            pers = None
    except Exception as e:
        print(e)
        pers = None
    if pers is not None:
        x.mentioned_person.add(pers)

In [ ]:
for x in Person.objects.all():
    x.written_name = x.legacy_id
    x.written_name_leven = x.legacy_id[:254]
    if len(x.legacy_id.split(' ')) > 1:
        x.forename = x.legacy_id.split(' ')[0]
        x.name = x.legacy_id.split(' ')[1]
    x.save()


In [ ]:
nlp = spacy.load('vfbr_places_model')

In [ ]:
for x in Person.objects.all():
    doc = nlp(x.written_name)
    if len(doc.ents) > 0:
        for ent in doc.ents:
            pl, _ = Place.objects.get_or_create(name=ent.text)
            x.belongs_to_place.add(pl)

In [ ]:
for x in Place.objects.all():
    if "(" in x.name and ")" in x.name:
        part_of_name = x.name.split('(')[1].split(')')[0].strip()
        part_of_place, _ = Place.objects.get_or_create(name=part_of_name)
        x.part_of = part_of_place
        x.save()

In [ ]:
nlp = spacy.load('vfbr_jobs_model')

In [ ]:
scheme, _ = SkosCollection.objects.get_or_create(name="profession")

In [ ]:
for x in Person.objects.all():
    doc = nlp(x.written_name)
    if len(doc.ents) > 0:
        for ent in doc.ents:
            job, _ = SkosConcept.objects.get_or_create(pref_label=ent.text)
            job.collection.add(scheme)
            x.profession.add(job)

In [ ]:
nlp = spacy.load('invs_nn')

In [ ]:
for x in Person.objects.all():
    doc = nlp(x.written_name)
    if len(doc.ents) > 0:
        name = " ".join([ent.text for ent in doc.ents])
        x.name = name
        x.save()

In [ ]:
nlp = spacy.load('invs_gender_binary')

In [ ]:
for x in Person.objects.all():
    doc = nlp(x.written_name)
    if doc.cats['male'] >= 0.6:
        x.gender = 'männlich'
    elif doc.cats['male'] <= 0.4:
        x.gender = 'weiblich'
    else:
        x.gender = 'anders'
    x.save()

In [ ]:
nlp = spacy.load('invs_vn')

In [ ]:
for x in Person.objects.all():
    doc = nlp(x.written_name)
    if len(doc.ents) > 0:
        name = " ".join([ent.text for ent in doc.ents]).strip()
        ws_nr = name.count(' ')
        if name == 'Veit [':
            x.forename = 'Veit [Vitus]'
        if ws_nr == 0:
            x.forename = name
        if ws_nr == 1:
            x.forename = name
        if ws_nr > 1:
            new_name = " ".join(name.split(' ')[:2])
            x.forename = new_name
        x.save()

In [ ]:
nlp = spacy.load('invs_fam')

In [ ]:
persons = Person.objects.filter(is_related_person__isnull=False)
persons.count()

In [ ]:
fam_scheme, _ = SkosCollection.objects.get_or_create(name="rel_type")

In [ ]:
for x in persons:
    doc = nlp(x.written_name)
    ents = doc.ents
    if len(ents) == 1:
        fam_rel_type = None
        perper = None
        try:
            inv = x.is_related_person.all()[0]
        except IndexError:
            inv = None
        try:
            main_person = inv.main_person.all()[0]
        except IndexError:
            main_person = None
        if main_person is not None:
            fam_rel_type, _ = SkosConcept.objects.get_or_create(pref_label="{}".format(ents[0]))
            fam_rel_type.collection.add(fam_scheme)
            perper, _ = PersonPerson.objects.get_or_create(
                source=x,
                target=main_person,
                rel_type=fam_rel_type
            )
            perper.save()
    else:
        pass

In [ ]:
PersonPerson.objects.all()[0].source.id